In [ ]:
# Autoreload imports
%load_ext autoreload
%autoreload 2

# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import plotly.express as px

from ecodynelec.parameter import Parameter  # Import the class to manipulate parameters
from ecodynelec.preprocessing.downloading import download #import the class to download entsoe data
from examples.mix_analysis.analysis_functions import format_data_0, get_metrics, plot_hourly_heatmap, plot_years, \
    plot_typical_days

# change plt font size
plt.rcParams.update({'font.size': 12})

In [ ]:
# EcoDynElec configuration
my_config = Parameter()
my_config.freq = "H"
my_config.ctry = ['AT', 'CH', 'DE', 'FR', 'IT']
#my_config.target = ['CH']  # You can put multiple targets here
# Enable this if you want to plot fig 6 (will impact the computation time - it took me 15min)
my_config.target = ['CH', 'AT', 'FR', 'DE','IT']
my_config.data_cleaning = True
my_config.sg_imports = True

# Dynamic impact for STEP
my_config.dynamic_impact = True
my_config.interpolated_stock = False # if False dynamic impact = Mix CH, if True storage data are interpolated from week to hour

# Indicate where to save generation data
my_config.path.generation = "./test_data/downloads/generations" 
# Indicate where to save exchange data
my_config.path.exchanges = "./test_data/downloads/exchanges/"
# Recommended config: include swiss grid residual (difference between swiss grid and entso-e data)
my_config.residual_global = True
# Recommended config: include real (2020-2022) and estimated (2016-2019) renewable production (using EcoDynElec-Enr-Model)
my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2024_completed.csv'
#my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2023_completed.csv'
# Years to study
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

In [ ]:
from examples.mix_analysis.analysis_functions import load_data

flows, prods, mixs, prod_impacts, impacts = load_data(my_config.target, my_config.freq, years,
                                                      savedir="./mix_analysis/results_local_all_enr/")

In [4]:
flows_CH = [flows[i]['CH'] for i in flows.keys()]
flows_CH=pd.concat(flows_CH)


flows_AT = [flows[i]['AT'] for i in flows.keys()]
flows_AT=pd.concat(flows_AT)


flows_DE = [flows[i]['DE'] for i in flows.keys()]
flows_DE=pd.concat(flows_DE)


flows_FR = [flows[i]['FR'] for i in flows.keys()]
flows_FR=pd.concat(flows_FR)


flows_IT = [flows[i]['IT'] for i in flows.keys()]
flows_IT=pd.concat(flows_IT)

In [5]:
flows_IT

,production,imports,exports
2016-01-01 00:00:00,19268.000000,3603.8920,167.910000
2016-01-01 01:00:00,18067.000000,3246.0800,150.810000
2016-01-01 02:00:00,17122.000000,3264.3928,93.320000
2016-01-01 03:00:00,16313.000000,3480.9678,114.100000
2016-01-01 04:00:00,16194.000000,3625.4790,114.750000
...,...,...,...
2024-12-31 19:00:00,24828.484894,5206.4604,14.636000
2024-12-31 20:00:00,21791.265137,5571.5590,16.220000
2024-12-31 21:00:00,19282.755524,6119.8760,0.226000
2024-12-31 22:00:00,17904.950592,6613.5010,15.459000


In [6]:
from examples.mix_analysis.analysis_functions import concatenate_and_format_data

# NOTE: Unless you use multiple targets (see section 0.), these dicts will contain only one element, corresponding to the target country
raw_productions_by_src, raw_consumptions_by_src, electricity_prod_mixs, electricity_mixs, producing_electricity_impacts, electricity_impacts = concatenate_and_format_data(my_config.target, years, flows, prods, mixs, prod_impacts, impacts)

In [7]:
# Sauvegarder les impacts d'électricité pour l'Italie (déjà existant)
electricity_impact_by_src_IT = electricity_impacts['IT']['raw_df']


# Sauvegarder les impacts d'électricité pour l'Autriche (AT)
electricity_impact_by_src_AT = electricity_impacts['AT']['raw_df']


# Sauvegarder les impacts d'électricité pour la Suisse (CH)
electricity_impact_by_src_CH = electricity_impacts['CH']['raw_df']


# Sauvegarder les impacts d'électricité pour la France (FR)
electricity_impact_by_src_FR = electricity_impacts['FR']['raw_df']


# Sauvegarder les impacts d'électricité pour l'Allemagne (DE)
electricity_impact_by_src_DE = electricity_impacts['DE']['raw_df']

In [8]:
electricity_impact_by_src_DE

,Mix_Other,Residual_Hydro_Water_Reservoir_AT,Residual_Hydro_Run-of-river_and_poundage_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,...,Hydro_Pumpage_AT,Hydro_Pumpage_CH,Hydro_Pumpage_DE,Hydro_Pumpage_FR,Hydro_Pumpage_IT,Energy_storage_AT,Energy_storage_CH,Energy_storage_DE,Energy_storage_FR,Energy_storage_IT
2016-01-01 00:00:00,9.480828,0.0,0.0,0.0,0.000495,0.0,0.0,0.007220,0.002496,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,9.635539,0.0,0.0,0.0,0.000314,0.0,0.0,0.004537,0.001597,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 02:00:00,9.825601,0.0,0.0,0.0,0.000150,0.0,0.0,0.002071,0.000773,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 03:00:00,8.924160,0.0,0.0,0.0,0.000267,0.0,0.0,0.003518,0.001372,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 04:00:00,8.954698,0.0,0.0,0.0,0.000382,0.0,0.0,0.004931,0.001963,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,13.805692,0.0,0.0,0.0,0.000953,0.0,0.0,0.036987,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-31 20:00:00,16.223881,0.0,0.0,0.0,0.000295,0.0,0.0,0.010525,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-31 21:00:00,17.320877,0.0,0.0,0.0,0.000241,0.0,0.0,0.008287,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-31 22:00:00,18.204968,0.0,0.0,0.0,0.000422,0.0,0.0,0.014345,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
tot_consumption_CH = raw_consumptions_by_src['CH']['df']


# Sauvegarder la consommation totale pour l'Autriche (AT)
tot_consumption_AT = raw_consumptions_by_src['AT']['df']


# Sauvegarder la consommation totale pour la France (FR)
tot_consumption_FR = raw_consumptions_by_src['FR']['df']

# Sauvegarder la consommation totale pour l'Allemagne (DE)
tot_consumption_DE = raw_consumptions_by_src['DE']['df']


# Sauvegarder la consommation totale pour l'Italie (IT)
tot_consumption_IT = raw_consumptions_by_src['IT']['df']

In [10]:
tot_consumption_IT

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,58.742504,1014.913475,704.638216,1327.485863,19149.887947,448.313918,22703.981922
2016-01-01 01:00:00,56.281714,925.795928,679.110998,1071.120723,17956.200514,473.759971,21162.269848
2016-01-01 02:00:00,73.008258,1010.544276,784.786151,780.807530,17053.976835,589.951202,20293.074253
2016-01-01 03:00:00,59.134831,1008.986648,685.510790,1203.681833,16229.093101,493.460503,19679.867707
2016-01-01 04:00:00,56.202080,1035.623307,661.376058,1365.601688,16110.321668,475.605517,19704.730318
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,288.905498,1996.695211,1039.092527,1481.210302,24819.970465,394.437744,30020.311746
2024-12-31 20:00:00,337.845932,1716.261386,1218.301729,1595.145289,21782.369801,696.680446,27346.604584
2024-12-31 21:00:00,342.586733,1828.153064,1309.893929,1730.230926,19282.640071,908.900864,25402.405586
2024-12-31 22:00:00,255.386805,2052.485371,1359.494483,1974.685889,17897.048723,963.891788,24502.993059


In [11]:
raw_consumption_by_src_IT = raw_consumptions_by_src['IT']['raw_df']


# Sauvegarder la consommation brute pour l'Autriche (AT)
raw_consumption_by_src_AT = raw_consumptions_by_src['AT']['raw_df']


# Sauvegarder la consommation brute pour la Suisse (CH)
raw_consumption_by_src_CH = raw_consumptions_by_src['CH']['raw_df']


# Sauvegarder la consommation brute pour la France (FR)
raw_consumption_by_src_FR = raw_consumptions_by_src['FR']['raw_df']


# Sauvegarder la consommation brute pour l'Allemagne (DE)
raw_consumption_by_src_DE = raw_consumptions_by_src['DE']['raw_df']

In [27]:
raw_consumption_by_src_CH

,Mix_Other,Residual_Hydro_Water_Reservoir_AT,Residual_Hydro_Run-of-river_and_poundage_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,...,Hydro_Pumpage_AT,Hydro_Pumpage_CH,Hydro_Pumpage_DE,Hydro_Pumpage_FR,Hydro_Pumpage_IT,Energy_storage_AT,Energy_storage_CH,Energy_storage_DE,Energy_storage_FR,Energy_storage_IT
2016-01-01 00:00:00,217.303313,0.0,0.0,0.0,21.181584,0.0,0.0,50.235541,10.507998,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,231.522017,0.0,0.0,0.0,22.385964,0.0,0.0,52.592291,11.200347,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 02:00:00,263.688455,0.0,0.0,0.0,25.777351,0.0,0.0,57.758458,13.043340,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 03:00:00,213.215571,0.0,0.0,0.0,21.758128,0.0,0.0,46.649423,11.009613,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 04:00:00,195.084527,0.0,0.0,0.0,20.434726,0.0,0.0,42.865245,10.326349,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,245.144436,0.0,0.0,0.0,18.042298,0.0,0.0,113.901281,0.000000,0.0,...,0.0,-7.168312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 20:00:00,329.463690,0.0,0.0,0.0,23.277973,0.0,0.0,135.139798,0.000000,0.0,...,0.0,-7.320235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 21:00:00,345.501882,0.0,0.0,0.0,22.988644,0.0,0.0,128.279796,0.000000,0.0,...,0.0,-7.186201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 22:00:00,436.238536,0.0,0.0,0.0,26.737569,0.0,0.0,147.688440,0.000000,0.0,...,0.0,-7.835859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [13]:
# Sauvegarder la production totale pour l'Italie (IT) - déjà existant
tot_production_IT = raw_productions_by_src['IT']['df']


# Sauvegarder la production totale pour l'Autriche (AT)
tot_production_AT = raw_productions_by_src['AT']['df']


# Sauvegarder la production totale pour la Suisse (CH)
tot_production_CH = raw_productions_by_src['CH']['df']


# Sauvegarder la production totale pour la France (FR)
tot_production_FR = raw_productions_by_src['FR']['df']


# Sauvegarder la production totale pour l'Allemagne (DE)
tot_production_DE = raw_productions_by_src['DE']['df']

In [14]:
tot_production_DE

,DE,sum
2016-01-01 00:00:00,48235.307274,48235.307274
2016-01-01 01:00:00,47771.397944,47771.397944
2016-01-01 02:00:00,47615.552137,47615.552137
2016-01-01 03:00:00,47392.680643,47392.680643
2016-01-01 04:00:00,46317.940983,46317.940983
...,...,...
2024-12-31 19:00:00,57190.071056,57190.071056
2024-12-31 20:00:00,54823.584851,54823.584851
2024-12-31 21:00:00,53564.440340,53564.440340
2024-12-31 22:00:00,53546.674238,53546.674238


In [15]:
production_IT = raw_productions_by_src['IT']['raw_df']


# Sauvegarder la production brute pour l'Autriche (AT)
production_AT = raw_productions_by_src['AT']['raw_df']


# Sauvegarder la production brute pour la Suisse (CH)
production_CH = raw_productions_by_src['CH']['raw_df']

# Sauvegarder la production brute pour la France (FR)
production_FR = raw_productions_by_src['FR']['raw_df']


# Sauvegarder la production brute pour l'Allemagne (DE)
production_DE = raw_productions_by_src['DE']['raw_df']

In [30]:
production_CH['Hydro_Pumped_Storage_CH'].loc['2023'].sum()

np.float64(11013434.571760574)

In [17]:
tot_electricity_mix_IT = electricity_mixs['IT']['df']


# Sauvegarder le mix électrique total pour l'Autriche (AT)
tot_electricity_mix_AT = electricity_mixs['AT']['df']


# Sauvegarder le mix électrique total pour la Suisse (CH)
tot_electricity_mix_CH = electricity_mixs['CH']['df']


# Sauvegarder le mix électrique total pour la France (FR)
tot_electricity_mix_FR = electricity_mixs['FR']['df']


# Sauvegarder le mix électrique total pour l'Allemagne (DE)
tot_electricity_mix_DE = electricity_mixs['DE']['df']

In [18]:
tot_electricity_mix_DE

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,0.000047,0.000817,0.962751,0.010880,1.879632e-05,0.025486,1.0
2016-01-01 01:00:00,0.000029,0.000485,0.962236,0.011339,8.929324e-06,0.025902,1.0
2016-01-01 02:00:00,0.000014,0.000193,0.966127,0.007251,1.975675e-06,0.026413,1.0
2016-01-01 03:00:00,0.000024,0.000412,0.957216,0.018354,4.133330e-06,0.023990,1.0
2016-01-01 04:00:00,0.000035,0.000639,0.954787,0.020460,6.223155e-06,0.024072,1.0
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.000167,0.000087,0.954546,0.008087,1.561190e-07,0.037112,1.0
2024-12-31 20:00:00,0.000051,0.000189,0.941141,0.015006,4.421643e-07,0.043613,1.0
2024-12-31 21:00:00,0.000042,0.000153,0.930502,0.022741,4.682559e-09,0.046561,1.0
2024-12-31 22:00:00,0.000062,0.000475,0.928224,0.022300,7.846937e-07,0.048938,1.0


In [19]:
tot_electricity_impact_IT = electricity_impacts['IT']['df']


# Sauvegarder l'impact électrique total pour l'Autriche (AT)
tot_electricity_impact_AT = electricity_impacts['AT']['df']


# Sauvegarder l'impact électrique total pour la Suisse (CH)
tot_electricity_impact_CH = electricity_impacts['CH']['df']


# Sauvegarder l'impact électrique total pour la France (FR)
tot_electricity_impact_FR = electricity_impacts['FR']['df']


# Sauvegarder l'impact électrique total pour l'Allemagne (DE)
tot_electricity_impact_DE = electricity_impacts['DE']['df']

In [20]:
tot_electricity_impact_DE

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,0.011051,0.010599,522.005743,0.428125,0.022491,9.480828,531.958837
2016-01-01 01:00:00,0.006976,0.005364,521.180094,0.454852,0.010709,9.635539,531.293534
2016-01-01 02:00:00,0.003247,0.002112,523.091545,0.298325,0.002361,9.825601,533.223191
2016-01-01 03:00:00,0.005601,0.004362,519.842101,0.775890,0.004955,8.924160,529.557069
2016-01-01 04:00:00,0.007914,0.006771,531.807142,0.844825,0.007415,8.954698,541.628765
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.044571,0.002054,301.140233,0.227339,0.000090,13.805692,315.219979
2024-12-31 20:00:00,0.014221,0.003110,261.525583,0.397019,0.000256,16.223881,278.164072
2024-12-31 21:00:00,0.011925,0.002480,258.718146,0.627823,0.000003,17.320877,276.681253
2024-12-31 22:00:00,0.015538,0.007297,230.432841,0.642467,0.000458,18.204968,249.303569


In [21]:
src_mapping = {'Mix_Other' : 'Other',
'Residual_Hydro_Run-of-river_and_poundage': 'Hydro run-of-river',
'Biomass' :'Biomass' ,
'Fossil_Coal-derived_gas': 'Gas',
'Fossil_Oil':'Oil',
'Geothermal' :'Geothermal',
'Hydro_Run-of-river_and_poundage' :'Hydro run-of-river',
'Marine': 'Other',
'Solar' : 'Solar',
'Residual_Hydro_Water_Reservoir' : 'Hydro lake',
'Residual_Other': 'Other',
'Fossil_Brown_coal/Lignite':'Coal',
'Fossil_Gas': 'Gas',
'Fossil_Oil_shale':'Oil',
'Hydro_Pumped_Storage' : 'Hydro storage',
'Hydro_Water_Reservoir' : 'Hydro lake',
'Nuclear' : 'Nuclear',
'Waste' : 'Waste',
'Fossil_Hard_coal':'Coal',
'Fossil_Peat':'Coal',
'Other_fossil': 'Other',
'Wind_Offshore' : 'Wind',
'Other_renewable': 'Other',
'Wind_Onshore' : 'Wind'}


def group_by_src(DF):
    # Supposons que df soit votre DataFrame
    df = DF.copy()
    df.columns = [col[:-3] if col != 'Mix_Other' else col for col in df.columns]
    df = df.groupby(df.columns, axis=1).sum()
    df = pd.concat([df, df[['Mix_Other']]], axis=1)
    df = df.rename(columns=src_mapping)
    df = df.groupby(df.columns, axis=1).sum()

    return df

In [25]:
Techno_impact_IT = group_by_src(electricity_impact_by_src_IT)


Techno_impact_AT = group_by_src(electricity_impact_by_src_AT)


Techno_impact_CH = group_by_src(electricity_impact_by_src_CH)

Techno_impact_FR = group_by_src(electricity_impact_by_src_FR)


Techno_impact_DE = group_by_src(electricity_impact_by_src_DE)

# Pour l'Italie (IT)
Techno_IT = group_by_src(raw_consumption_by_src_IT)


Techno_AT = group_by_src(raw_consumption_by_src_AT)


Techno_CH = group_by_src(raw_consumption_by_src_CH)


Techno_FR = group_by_src(raw_consumption_by_src_FR)


Techno_DE = group_by_src(raw_consumption_by_src_DE)

C:\Users\PC Jules\AppData\Local\Temp\ipykernel_14588\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_14588\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_14588\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_14588\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_14588\653172070.py:31: FutureWarning: DataFrame.groupby with axis

In [23]:
Techno_IT

,Biomass,Coal,Energy_storage,Gas,Geothermal,Hydro lake,Hydro run-of-river,Hydro storage,Hydro_Pumpage,Nuclear,Oil,Other,Solar,Waste,Wind
2016-01-01 00:00:00,296.287861,1996.480559,0.000000,5338.834586,701.921270,253.448821,1358.262045,67.930114,0.000000,1950.385383,524.238361,10219.410043,1.010254,59.202998,384.883545
2016-01-01 01:00:00,294.024536,1993.858703,0.000000,4785.405281,701.910989,175.946060,1266.411112,35.731193,0.000000,1690.424593,534.055843,9685.721603,0.013107,56.645616,415.881184
2016-01-01 02:00:00,305.313192,2036.395251,0.000000,4585.163800,702.478423,185.320334,1199.686133,35.785170,0.000000,1522.444136,471.793234,9400.340333,0.012582,53.410904,384.881963
2016-01-01 03:00:00,294.747704,1930.594018,0.000000,4240.562616,703.608534,185.997246,1186.879028,39.223350,0.000000,1869.174439,431.862254,8877.969589,0.011275,57.243399,355.454759
2016-01-01 04:00:00,294.239874,1927.584724,0.000000,4252.933677,702.595446,196.243321,1210.290055,44.191703,0.000000,2021.049737,413.615305,8712.692279,0.016340,54.221151,350.662222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,710.020993,562.368410,0.000000,18309.471423,601.715021,797.380611,2840.111506,591.633889,-3.352363,1957.374570,33.278393,2571.382776,0.313763,344.577202,1098.473296
2024-12-31 20:00:00,731.494575,543.533132,0.000000,15740.537149,601.894274,639.999335,2566.120744,330.834545,-3.505204,2102.071711,35.007406,3104.947069,0.297975,355.037576,1295.014745
2024-12-31 21:00:00,742.229362,540.533260,0.000000,13810.920816,601.766033,535.066986,2251.568088,308.990859,-3.669455,2239.970046,36.034742,3464.663370,0.288238,375.245487,1407.698617
2024-12-31 22:00:00,739.143096,508.842763,0.000000,12717.242985,600.986989,599.928164,2315.361767,276.840919,-3.576412,2327.470609,36.728008,3402.084998,0.237295,378.771227,1566.822441


## Sauvegarde des DataFrames au format parquet.gz

In [24]:
base_dir = r'C:\Users\PC Jules\Desktop\HydraulicDam_emissions\data\ecodynelec'

#

folders = {
    'consumptions': {
        'raw_consumption_by_src_AT': raw_consumption_by_src_AT,
        'raw_consumption_by_src_CH': raw_consumption_by_src_CH,
        'raw_consumption_by_src_DE': raw_consumption_by_src_DE,
        'raw_consumption_by_src_FR': raw_consumption_by_src_FR,
        'raw_consumption_by_src_IT': raw_consumption_by_src_IT,
        'tot_consumption_AT': tot_consumption_AT,
        'tot_consumption_CH': tot_consumption_CH,
        'tot_consumption_DE': tot_consumption_DE,
        'tot_consumption_FR': tot_consumption_FR,
        'tot_consumption_IT': tot_consumption_IT
    },
    'electricity_impacts': {
        'electricity_impact_AT': tot_electricity_impact_AT,
        'electricity_impact_CH': tot_electricity_impact_CH,
        'electricity_impact_DE': tot_electricity_impact_DE,
        'electricity_impact_FR': tot_electricity_impact_FR,
        'electricity_impact_IT': tot_electricity_impact_IT,
        'electricity_impact_by_src_AT': electricity_impact_by_src_AT,
        'electricity_impact_by_src_CH': electricity_impact_by_src_CH,
        'electricity_impact_by_src_DE': electricity_impact_by_src_DE,
        'electricity_impact_by_src_FR': electricity_impact_by_src_FR,
        'electricity_impact_by_src_IT': electricity_impact_by_src_IT
    },
    'electricity_mixs': {
        'electricity_mix_AT': tot_electricity_mix_AT,
        'electricity_mix_CH': tot_electricity_mix_CH,
        'electricity_mix_DE': tot_electricity_mix_DE,
        'electricity_mix_FR': tot_electricity_mix_FR,
        'electricity_mix_IT': tot_electricity_mix_IT
    },
    'flows': {
        'flows_AT': flows_AT,
        'flows_CH': flows_CH,
        'flows_DE': flows_DE,
        'flows_FR': flows_FR,
        'flows_IT': flows_IT
    },
    'productions': {
        'production_AT': production_AT,
        'production_CH': production_CH,
        'production_DE': production_DE,
        'production_FR': production_FR,
        'production_IT': production_IT,
        'tot_production_AT': tot_production_AT,
        'tot_production_CH': tot_production_CH,
        'tot_production_DE': tot_production_DE,
        'tot_production_FR': tot_production_FR,
        'tot_production_IT': tot_production_IT
    },
    'technologies': {
        'Techno_impact_AT': Techno_impact_AT,
        'Techno_impact_CH': Techno_impact_CH,
        'Techno_impact_DE': Techno_impact_DE,
        'Techno_impact_FR': Techno_impact_FR,
        'Techno_impact_IT': Techno_impact_IT,
        'technologies_AT': Techno_AT,
        'technologies_CH': Techno_CH,
        'technologies_DE': Techno_DE,
        'technologies_FR': Techno_FR,
        'technologies_IT': Techno_IT
    }
}

for folder_name, dataframes in folders.items():
    folder_path = os.path.join(base_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    for file_name, df in dataframes.items():
        file_path = os.path.join(folder_path, f'{file_name}.parquet.gz')
        df.to_parquet(file_path, compression='gzip')
        print(f'Saved: {file_path}')

print('\nToutes les données ont été sauvegardées avec succès!')

NameError: name 'Techno_impact_IT' is not defined